In [ ]:
import cv2
print(cv2.__version__)


In [1]:
import cv2

video_path = 'input_video.mp4'
cap = cv2.VideoCapture(video_path)

ret, frame = cap.read()
if not ret:
    print("Failed to read video")
    cap.release()
    exit()

scale_percent = 50 
width = int(frame.shape[1] * scale_percent / 100)
height = int(frame.shape[0] * scale_percent / 100)
dim = (width, height)

resized_frame = cv2.resize(frame, dim, interpolation=cv2.INTER_AREA)
roi = cv2.selectROI("Select ROI", resized_frame, fromCenter=False, showCrosshair=False)
cv2.destroyWindow("Select ROI")


In [5]:

output_path = 'tracked_output.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
fps = cap.get(cv2.CAP_PROP_FPS)
frame_size = (frame.shape[1], frame.shape[0])

out = cv2.VideoWriter(output_path, fourcc, fps, frame_size)


In [7]:

tracker = cv2.TrackerCSRT_create()

x, y, w, h = roi
x = int(x * 100 / scale_percent)
y = int(y * 100 / scale_percent)
w = int(w * 100 / scale_percent)
h = int(h * 100 / scale_percent)

roi_scaled = (x, y, w, h)

tracker.init(frame, roi_scaled)


In [9]:
# trail_points = []
while True:
    ret, frame = cap.read()
    if not ret:
        print("I did not get the Frame")
        break
    
    success, box = tracker.update(frame)
    if success:
        x, y, w, h = map(int, box)
        # center = (x + w // 2, y + h // 2)
        # trail_points.append(center)
        # cv2.rectangle(img, (x - padding, y - text_h - padding),(x + text_w + padding, y + padding),bg_color, -1)
        cv2.rectangle(frame, (x-2, y - 40), (x + 150, y), (0,252,0), -1)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0,252,0), 3)
        b, g, r = frame[y-100, x]
        bgColor = (int(b), int(g), int(r))  # Convert each to native Python int
        cv2.putText(frame, "Tracking", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, bgColor, 2)
        # for i in range(1, len(trail_points)):
        #     cv2.line(frame, trail_points[i - 1], trail_points[i], (255, 0, 0), 2)
    else:
        cv2.putText(frame, "Lost", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 1)

    display_frame = cv2.resize(frame, (width, height))
    out.write(frame)

    cv2.imshow("CSRT Tracking", display_frame)

    if cv2.waitKey(30) & 0xFF == 27: 
        break
cap.release()
cv2.destroyAllWindows()
out.release()
print("Tracking video saved as:", output_path)

I did not get the Frame
Tracking video saved as: tracked_output.mp4
